<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/qchem_500_MindTheGap_template/qchem_500_MindTheGap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import hf

The objective of this challenge is to find the ground state energy as well as the energy of the first excited state of Hydrogen. We can represent the $H_2$ molecule. We can represent the $H_2$ molecule in dirac notation as a state:
$$|H_{2, init}\rangle = \begin{pmatrix} 1 \\ 1 \\ 0 \\ 0 \end{pmatrix}$$
where each 1 represents an electron.

Since we are dealing with the representation of a molecule, the number of particles must be preserved, so we must use particle preserving gates, namely [Givens rotations](https://pennylane.ai/qml/demos/tutorial_givens_rotations.html).

Given the Hamiltonian of $H_2$, $\hat{H}$, the energy of the ground state can be found using DoubleExcitationGates as:
$$E(\theta) = \langle \Psi_0(\theta) | \hat{H} | \Psi_0(\theta) \rangle = \langle H_{2, init}| G_2(\theta)^\dagger \; \hat{H} \; G_2(\theta) |H_{2, init}\rangle$$  

One just needs to implement [VQE](https://pennylane.ai/qml/demos/tutorial_vqe.html) to find the correct angle $\theta$ that characterizes the ground state.

In [ ]:
def ground_state_VQE(H):
    """Perform VQE to find the ground state of the H2 Hamiltonian.
    Args:
        - H (qml.Hamiltonian): The Hydrogen (H2) Hamiltonian
    Returns:
        - (float): The ground state energy
        - (np.ndarray): The ground state calculated through your optimization routine
    """

    # QHACK #
    
    # Initialize thhe fevice
    num_qubits = len(H.wires)
    qubits = H.wires
    num_param_sets = (2 ** num_qubits) - 1
    dev = qml.device("default.qubit", wires=num_qubits)

    # Initialize the energy, theta and hi state
    energy = 0
    hi = np.array([1, 1, 0, 0])
    theta = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(1))

    # Create circuit for VQE
    def circuit(theta):
        qml.DoubleExcitation(theta[0], wires=qubits)

    # Define cost function and a function to return the ground_state
    @qml.qnode(dev)
    def cost_fn(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.expval(H)
    @qml.qnode(dev)
    def get_ground_state(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.state()

    # Define the optimizer, parameters of the VQE and lists to keep track of parameter and results at every iteration
    opt = qml.optimize.AdamOptimizer(0.1)
    max_iterations = 200
    conv_tol = 1e-06
    energy = [cost_fn(theta)]
    theta_col = [theta]
    states = [get_ground_state(theta)]

    # Perform VQE to find the angle theta
    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_fn, theta)

        energy.append(cost_fn(theta))
        theta_col.append(theta)
        states.append(get_ground_state(theta))

        conv = np.abs(energy[-1] - prev_energy)

        if n % 2 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} HH")

        if conv <= conv_tol:
            break
    print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} HH")
    print("\n" f"Optimal value of the circuit parameters = {theta_col[-1]}")
    print("\n" f"Ground-state state vector = {states[-1]}")

    # Return the ground state energy and the ground state
    return energy[-1], states[-1]
    # QHACK #


As illustrated in the problem pdf, given the ground state $|\lambda_0\rangle = |\Psi_0(\theta^*)\rangle$ and the Ground state Hamiltonian $\hat{H}$ we can calculate the excited state Hamiltonian as:
$$\hat{H}_1=\hat{H} + \beta |\lambda_0\rangle \langle \lambda_0 | = (E_0 + \beta)  |\lambda_0\rangle \langle \lambda_0 |  + \sum_i E_ i |\lambda_i\rangle \langle \lambda_i | $$

This can be done using the decomposing the matrix $(E_0 + \beta)  |\lambda_0\rangle \langle \lambda_0 |$ into observables and coefficients

Then one can calculate the Ground state energy as done before and perfor VQE to find the new correct angle $\theta$. The only difference is that, in this case, since we are working with the first excited states, one of the electrons of the molecules will be in its second energy level so we should use Single Excitation Givens gates.

$$E_1(\theta) = \langle \Psi_1(\theta) | \hat{H}_1 | \Psi_1(\theta) \rangle = \langle H_{2, init}| G_1(\theta)^\dagger \; \hat{H}_1 \; G_1(\theta) |H_{2, init}\rangle$$  

In [ ]:
def create_H1(ground_state, beta, H):
    """Create the H1 matrix, then use `qml.Hermitian(matrix)` to return an observable-form of H1.
    Args:
        - ground_state (np.ndarray): from the ground state VQE calculation
        - beta (float): the prefactor for the ground state projector term
        - H (qml.Hamiltonian): the result of hf.generate_hamiltonian(mol)()
    Returns:
        - (qml.Observable): The result of qml.Hermitian(H1_matrix)
    """

    # QHACK #
    # Define function to decompose hamiltonian into observables and coefficients
    from functools import reduce
    from itertools import product
    from operator import matmul
    def decompose_hamiltonian(H):
        N = int(np.log2(len(H)))
        paulis = [qml.Identity, qml.PauliX, qml.PauliY, qml.PauliZ]

        obs = []
        coeffs = []

        for term in product(paulis, repeat=N):
            matrices = [i._matrix() for i in term]
            coeff = np.trace(reduce(np.kron, matrices) @ H) / (2 ** N)

            if not np.allclose(coeff, 0):
                coeffs.append(coeff)

                if not all(t is qml.Identity for t in term):
                    obs.append(reduce(matmul, [t(i) for i, t in enumerate(term) if t is not qml.Identity]))
                else:
                    obs.append(reduce(matmul, [t(i) for i, t in enumerate(term)]))

        return coeffs, obs

    # Get new coefficients and new observables to add to H
    gs = ground_state.reshape(-1, 1)
    new_coefficients, new_observables = decompose_hamiltonian(beta * (gs @ gs.T))

    # Add new observables and new coefficients
    new_coefficients = np.tensor(new_coefficients)
    new_coefficients = np.concatenate((new_coefficients, H.terms[0]))
    new_observables += H.terms[1]
    
    # Create new H1 hamiltonian out of new lists of coefficients and observables
    H1 = qml.Hamiltonian(new_coefficients, new_observables)
    return H1
    # QHACK #

In [ ]:
def excited_state_VQE(H1):
    """Perform VQE using the "excited state" Hamiltonian.
    Args:
        - H1 (qml.Observable): result of create_H1
    Returns:
        - (float): The excited state energy
    """

    # QHACK #
    # Initialize thhe fevice
    num_qubits = len(H1.wires)
    qubits = H1.wires
    num_param_sets = (2 ** num_qubits) - 1
    dev = qml.device("default.qubit", wires=num_qubits)

    # Initialize the energy, theta and hi state
    energy = 0
    hi = np.array([1, 1, 0, 0])
    theta = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(1))

    # Create circuit for VQE
    def circuit(theta):
        qml.SingleExcitation(theta[0], wires=[1, 2])

    # Define cost function and a function to return the ground_state
    @qml.qnode(dev)
    def cost_fn(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.expval(H1)
    @qml.qnode(dev)
    def get_ground_state(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.state()

    # Define the optimizer, parameters of the VQE and lists to keep track of parameter and results at every iteration
    opt = qml.optimize.MomentumOptimizer()
    max_iterations = 300
    conv_tol = 1e-05
    energy = [cost_fn(theta)]
    theta_col = [theta]
    states = [get_ground_state(theta)]

    # Perform VQE to find the angle theta
    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_fn, theta)

        energy.append(cost_fn(theta))
        theta_col.append(theta)
        states.append(get_ground_state(theta))

        conv = np.abs(energy[-1] - prev_energy)

        if n % 2 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} HH")

        if conv <= conv_tol:
            break
    print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} HH")
    print("\n" f"Optimal value of the circuit parameters = {theta_col[-1]}")
    print("\n" f"Ground-state state vector = {states[-1]}")

    # Return the ground state energy and the ground state
    return energy[-1]
    # QHACK #

# Here we check the inputs of 1.in

In [ ]:
coord = 0.6614
symbols = ["H", "H"]
geometry = np.array([[0.0, 0.0, -coord], [0.0, 0.0, coord]], requires_grad=False)
mol = hf.Molecule(symbols, geometry)

H = hf.generate_hamiltonian(mol)()
E0, ground_state = ground_state_VQE(H)

beta = 15.0 + E0
H1 = create_H1(ground_state, beta, H)
E1 = excited_state_VQE(H1)

print(f"Ground state energy E_0 = {E0.item()}")
print(f"First excited state energy E_0 = {E1.item()}")